# 신경망 모델 구성하기
- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/ 모듈(module)로 구성되어 있음
- torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소 제공
- Pytorch의 모든 모듈은 nn.Module의 하위 클래스(suclass)임
- 신경망은 다른 모듈(계층; layer)로 구성된 모듈임

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
# 학습을 위한 장치 얻기
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {} device".format(device))

Using cuda device


# 클래스 정의하기
- 신경망 모델을 nn.Module의 하위 클래스로 정의
- __init__에서 신경망 계층들을 초기화
- nn.Module을 상속받는 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산 을 구함

In [4]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1,28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print("Predicted class: {}".format(y_pred))

Predicted class: tensor([7], device='cuda:0')


# 모델 계층(layer)


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# nn.Flatten 인접한 범위의 텐서를 병합함
# https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# nn.Linear 
# 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환을 적용하는 모듈
layer1 = nn.Linear(in_features=28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# nn.ReLU
# 비선형 활성화(activation)은 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만듦
# 비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 도움
print("Before ReLU: {}".format(hidden1))
hidden1 = nn.ReLU()(hidden1)
print("After ReLU: {}". format(hidden1))


Before ReLU: tensor([[-0.6282, -0.2842, -0.1933, -0.2956, -0.5710,  0.3113,  0.5167,  0.1898,
          0.0178, -0.1124, -0.0262,  0.1700, -0.1435, -0.0337,  0.5706, -0.1338,
         -0.1359, -0.7249,  0.7037,  0.0752],
        [-0.3328, -0.2182, -0.0929, -0.5507, -0.4204,  0.6621,  0.4149,  0.2524,
         -0.1947,  0.1768,  0.2076,  0.0795,  0.1065,  0.1633,  0.3660,  0.4336,
         -0.0560, -0.3822,  0.4831, -0.2157],
        [-0.1219, -0.2587,  0.1558, -0.3422, -0.4444,  0.3365,  0.3952,  0.3666,
         -0.0632, -0.0164, -0.1468,  0.3699, -0.0467,  0.4358,  0.3152,  0.2505,
         -0.3168, -0.3666,  0.6077, -0.0438]], grad_fn=<AddmmBackward0>)
After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3113, 0.5167, 0.1898, 0.0178,
         0.0000, 0.0000, 0.1700, 0.0000, 0.0000, 0.5706, 0.0000, 0.0000, 0.0000,
         0.7037, 0.0752],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6621, 0.4149, 0.2524, 0.0000,
         0.1768, 0.2076, 0.0795, 0.1065, 0.1633, 0.3660

In [11]:
# nn.Sequential
# 순서를 갖는 모듈의 컨테이너
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [12]:
# nn.Softmax
# 신경망의 마지막 선형 계층은 nn.softmax 모듈에 전달될 logits을 반환
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# 모델 매개변수
- 신경망 내부의 많은 계층들은 매개변수화 됨
- nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적됨

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
  print("Layer: {} | Size : {} | Values : {} \n".format(name, param.size(), param[:2]))

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0344,  0.0094,  0.0208,  ...,  0.0007, -0.0299,  0.0205],
        [-0.0130, -0.0254,  0.0311,  ...,  0.0332,  0.0171, -0.0220]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0312,  0.0129], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0385,  0.0330,  0.0437,  ..., -0.0023,  0.0070, -0.0319],
        [-0.0294,  0.0407,  0.0284,  ...,  0.0320,  0.0146, -0.0253]],
       device='cuda:0', grad_f